In [ ]:
import os
import sys
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch_harmonics as th
import torch_harmonics.distributed as dist
import xarray as xr
import xesmf as xe

module_path = os.path.abspath(os.path.join('..', 'MultiThread_Model'))
if module_path not in sys.path:
    sys.path.append(module_path)
from subs1_utils import *


warnings.filterwarnings("ignore")

# Model Variables
In the following cell you can set the values of the variables relevant to the model. The details of each variable are included in the README. In most cases it is only necessary to set values for the standard variables.

In [ ]:
# Define Spectral Truncation Desired and Consistent Gausian Grid.
zw = 63
kmax = 26

In [ ]:
# Check value for kmax.
if kmax != 11 and kmax != 26:
    raise Exception("Unexpected value for kmax: " + str(kmax))

# Check value for zw.
# Afterwards, set mw, jmax, and imax values based on
# the value given to zw.
match zw:
    case 42:
        mw = zw
        jmax = 64
        imax = 128
    case 63:
        mw = zw
        jmax = 96
        imax = 192
    case 124:
        mw = zw
        jmax = 188
        imax = 376
    case _:
        raise Exception("Unexpected value for zw: " + str(zw))

In [ ]:
# Create and return a path in which to save the preprocess output files.
preprocess_path = set_preprocess_path(zw, kmax)

# Setup necessary element for interpolation onto model Gaussian Grid.
cost_lg, wlg, lats, lons, vsht, dsht, disht, dvsht, divsht = \
    set_spectral_transforms(jmax, imax, mw, zw)
dlatlon = xr.Dataset({"lat": lats, "lon": lons})

In [ ]:
# Grab Topography Data.
url_topo = 'http://research.jisao.washington.edu/data_sets/elevation/elev.0.75-deg.nc'
ds_topo = xr.open_dataset(url_topo + '#mode=bytes', decode_times = False) # adding #mode=bytes because netcdf4 non-opendap URLrequeriment
del ds_topo['time']
data = ds_topo.data.squeeze()
data

In [ ]:
lon = np.linspace(-np.pi, np.pi, data.shape[1])
lat = np.linspace(np.pi/2., -np.pi/2., data.shape[0])
Lon, Lat = np.meshgrid(lon, lat)

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection='mollweide')
cs = ax.pcolormesh(Lon, Lat, data, cmap='turbo')
ax.set_title("Elevation map 0.75◦")
ax.grid(True)
ax.set_xticklabels([])
ax.set_yticklabels([])
fig.colorbar(cs, ax=ax, shrink=0.75)
plt.show()

In [ ]:
regridder_topog = xe.Regridder(data, dlatlon, 'bilinear')

In [ ]:
topog_gg = regridder_topog(data)*9.8

In [ ]:
topog_gg_dev = torch.from_numpy(np.where(topog_gg < 0.0, 0.0, topog_gg))
coeffs = dsht(topog_gg_dev)
topog_gg_r = disht(coeffs)

In [ ]:
# Write out the spectral coefficients for topography to be read by
# the AGCM as a pickle file.
torch.save(coeffs, preprocess_path+'topog.spectral.pt')

In [ ]:
# Now generature the vertical strucuture of temperature
# will neeed model vertical structure (si(kmax)).

delsig, si, sl, sikap, slkap, cth1, cth2, r1b, r2b = bscst(kmax)

temp_spec = torch.from_numpy(np.zeros((12, kmax, zw, mw)))
temp_gg = np.zeros((12, kmax, jmax, imax))

# Radiative equilibrium temperature
#   varying surface temperature decreasing with
#   height with a lapse rate dTe/dz approx = -rlaps degrees/m
#   to a stratospheric temperature of tstrat.

rlaps=6.8 * 1.0e-03
h0 = 8.2e+03
tstrat = 205.0 # Fixed stratospheric temperature.
deltemp = 60.0 # pole_to_equator temperature difference.
deltheta = 10.0
kappa = 2.0 / 7.0

# Set Radiative Temperature Profile here.
# Need to use topography to reduce temperature based on altitude.
# Calculate the radiative surface temperature as a function of latitude.
temp_gg = np.zeros((kmax, jmax, imax))
sinlat = np.sin(lats * np.pi / 180.0)
coslat = np.cos(lats * np.pi / 180.0)

for j in range(jmax):
    for k in range(kmax):
        temp_gg[k, j, :] = (
            315.0 - deltemp*sinlat[j]*sinlat[j]
            - deltheta*np.log(sl[k])*coslat[j]*coslat[j])*(sl[k]**kappa)

temp_gg = np.where(temp_gg < 200.0, 200.0, temp_gg)

In [ ]:
Lon, Lat = np.meshgrid(lons, lats)
plt.pcolormesh(Lon, Lat, temp_gg[kmax-1, :, :], cmap='bwr')
plt.colorbar()

In [ ]:
Lon, Zlev = np.meshgrid(lons, sl)
plt.pcolormesh(Lon, Zlev, temp_gg[:, 48, :], cmap='bwr')
plt.colorbar()

In [ ]:
Lat, Zlev = np.meshgrid(lats, -sl)
plt.pcolormesh(Lat, Zlev, temp_gg[:, :, 96], cmap='bwr')
plt.colorbar()

In [ ]:
temp_coeffs = dsht(torch.from_numpy(temp_gg))

In [ ]:
torch.save(temp_coeffs, preprocess_path+'temp.spectral_RadiativeEquilibrium.pt')
torch.save(temp_gg, preprocess_path+'temp.ggrid_RadiativeEquilibrium.pt')

In [ ]:
ps = np.zeros((jmax, imax))
lnps_coeffs = dsht(torch.from_numpy(ps))
torch.save(lnps_coeffs, preprocess_path+'lnps.spectral_RadiativeEquilibrium.pt')
print(['global mean lnps', lnps_coeffs[0, 0]])

In [ ]:
lnps_coeffs

In [ ]:
# Following Held & Suarez the Newtonian Damping Coefficients are
# latitudinally Dependent. Here we produce those coeffiencients
# and convert into spectral form.
ka = 1.0 / (40.0*86400.0)
ks = 1 / (4.0*86400.0)
slb = 0.7
vert = (sl-slb) / (1.0-slb)
vert = np.where(vert < 0.0, 0.0, vert)
kt = np.zeros((kmax, jmax, imax))
for j in range(jmax):
    for k in range(kmax):
        kt[k, j, :] = ka + (ks - ka)*vert[k]*(coslat[j]**4.0)

cooling_coeffs = dsht(torch.from_numpy(kt))
torch.save(cooling_coeffs, preprocess_path+'cooling.spectral_RadiativeEquilibrium.pt')
torch.save(kt, preprocess_path+'cooling.ggrid_RadiativeEquilibrium.pt')

In [ ]:
vert

In [ ]:
Lat, Zlev = np.meshgrid(lats, -sl)
plt.pcolormesh(Lat, Zlev, kt[:, :, 96]*86400, cmap='bwr')
plt.colorbar()